In [ ]:
from pysr import PySRRegressor
import sys
import commonutils
from commonutils import ModelResults

import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
#from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from dataclasses import dataclass
#import prettyprinter as pp

from sklearn.cross_decomposition import PLSRegression
import warnings
import sys

from sklearn import preprocessing

from copy import deepcopy
import pickle

import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
import sys
sys.path.append("./CLossLr")
import customlosslr as clr

from commonutils import ModelsStore
warnings.simplefilter("ignore")

from commonfuncsforcli import *

In [ ]:
selected_basisset = ["MINIX"]
selected_functional = ["PBE"]
Y_perset = {}
X_perset = {}


print("Selected functional: ", selected_functional)
print(" Selected basis set: ", selected_basisset)


basicfeattouse = ["Potential_Energy",\
                    "Kinetic_Energy",\
                    "FINAL_SINGLE_POINT_ENERGY",\
                    "Dispersion_correction",\
                    "E(C)",\
                    "E(X)",\
                    "Two_Electron_Energy",\
                    "Nuclear_Repulsion",\
                    "One_Electron_Energy"]

labelname = "label"

howmanydifs = 3
allvalues_perset = pickle.load(open("./data/allvalues_perset.p", "rb"))
fullsetnames = pickle.load(open("./data/fullsetnames.p", "rb"))
supersetnames = pickle.load(open("./data/supersetnames.p", "rb"))
methods = pickle.load(open("./data/methods.p", "rb"))
functionals = pickle.load(open("./data/functionals.p", "rb"))
basis_sets = pickle.load(open("./data/basis_sets.p", "rb"))

featuresvalues_perset = {}
for setname in fullsetnames:
    Y_perset[setname] = []
    X_perset[setname] = {}
    featuresvalues_perset[setname] = []
    for val in allvalues_perset[setname]:
        #print("Label value: ", val[labelname])
        Y_perset[setname].append(val[labelname])
        featuresvalues_perset[setname].append({})
        for k in val:
            if k.find("energydiff") != -1:
                torm = k.replace("energydiff", "")
                for f in val[k]:
                    tocheck = f.replace(torm, "")
                    if tocheck in basicfeattouse:
                        keytouse = f.replace("-", "_")
                        keytouse = keytouse.replace("(", "")
                        keytouse = keytouse.replace(")", "")
                        featuresvalues_perset[setname][-1][keytouse] = val[k][f]

featuresnames = set()
for setname in fullsetnames:
    if len(featuresvalues_perset[setname]) != len(Y_perset[setname]):
        print("Error: len(featuresvalues_perset[", setname, "]) != len(Y_perset[", setname, "])")
        print("  ", len(featuresvalues_perset[setname]), " != ", len(Y_perset[setname]))
        
    #print("Set: ", setname, " with ", len(featuresvalues_perset[setname]), " entries")
    #print("Label len(Y_perset[setname]): ", len(Y_perset[setname]))
    for i in range(len(featuresvalues_perset[setname])):
        for f in featuresvalues_perset[setname][i]:
            for func in selected_functional:
                for basis in selected_basisset:
                    if f.startswith(func+"_"+basis):
                        if f not in featuresnames:
                            featuresnames.add(f)

featuresnames = list(featuresnames)
featuresnames.sort()
print("Using features: ")
for f in featuresnames:
    print("  ", f)

for setname in fullsetnames:
    for f in featuresnames:
        X_perset[setname][f] = []

for setname in fullsetnames:
    for i in range(len(featuresvalues_perset[setname])):
        for f in featuresnames:
            for sf in featuresnames:
                if f == sf:
                    X_perset[setname][f].append(featuresvalues_perset[setname][i][sf])
            

toreplace = {"Dispersion_correction": "DC",
                "FINAL_SINGLE_POINT_ENERGY": "FSPE",
                "Kinetic_Energy": "KE",
                "Nuclear_Repulsion": "NR",
                "One_Electron_Energy": "OEE",
                "Potential_Energy": "PE",
                "Two_Electron_Energy": "TEE"} 

Xnew = {}
for setname in fullsetnames:
    Xnew[setname] = {}
    for f in X_perset[setname]:
        newf = f
        for k in toreplace:
            newf = newf.replace(k, toreplace[k])
        if newf != f:
            Xnew[setname][newf] = deepcopy(X_perset[setname][f])
            
X_perset = {}
X_perset = deepcopy(Xnew)
featuresnames = list(X_perset[fullsetnames[0]].keys())

# chck diemnsions
for setname in fullsetnames:
    #print("Set: ", setname)
    #print("  len(Y_perset[setname]): ", len(Y_perset[setname]))
    for f in featuresnames:
        #print("   Feature: ", f, " len(X_perset[setname][f]): ", len(X_perset[setname][f]))
        if len(X_perset[setname][f]) != len(Y_perset[setname]):
            print("Set: ", setname)
            print("Error: len(X_perset[", setname, "][", f, "]) != len(Y_perset[", setname, "])")
            print("  ", len(X_perset[setname][f]), " != ", len(Y_perset[setname]))

print("Using features: ")
for f in featuresnames:
    print("  ", f)


In [ ]:
# scatterplot of features vs label
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

for setname in supersetnames:
    for f in featuresnames:
        if "FSPE" in f:
            #plt.figure()
            #plt.scatter(X_perset[setname][f], Y_perset[setname])
            #plt.xlabel(f)
            #plt.ylabel(labelname)
            #plt.title("Set: "+setname+" Feature: "+f)
            #plt.show()
            mse = mean_squared_error(Y_perset[setname], X_perset[setname][f])
            rmse = np.sqrt(mse)
            r2 = r2_score(Y_perset[setname], X_perset[setname][f])
            mape = mean_absolute_percentage_error(Y_perset[setname], X_perset[setname][f])
            print("Set: ", setname, " Feature: ", f)
            print("  RMSE: ", rmse)
            print("    R2: ", r2)
            print("  MAPE: ", mape)
        #plt.savefig("scatter_"+setname+"_"+f+".png")
        #plt.close()

In [ ]:
from pysr import PySRRegressor
testlabel = "PBE_MINIX_FSPE"
metrics = {}
for setname in supersetnames:   
    print("Working on set: ", setname)
    metrics[setname] = {}
    mape_loss_string = "loss(prediction, target) = abs(prediction - target) / (abs(target) + 1e-9)"
    model = PySRRegressor(
        maxsize=20,
        niterations=40,  # < Increase me for better results
        binary_operators=["+", "*", "-"],
        unary_operators=[
          "cos",
          "exp",
          "sin",
          "inv(x) = 1/x"
          # ^ Custom operator (julia syntax)
        ],
        extra_sympy_mappings={"inv": lambda x: 1 / x},
        # ^ Define operator for SymPy as well
        elementwise_loss="loss(prediction, target) = (prediction - target)^2",
        # ^ Custom loss function (julia syntax)
        #elementwise_loss=mape_loss_string,
        )

    X = np.zeros((len(Y_perset[setname]), len(featuresnames)-1))
    cont = 0
    for i in range(len(featuresnames)):
        if "FSPE" not in featuresnames[i]:
            X[:, cont] = np.array(X_perset[setname][featuresnames[i]])
            cont += 1
    model.fit(X, Y_perset[setname])
    equations = model.equations_
    best_equation = equations.iloc[-1]
    #print("All discovered equations:")
    #print(equations[['equation', 'loss', 'complexity', 'score']].to_string())
    #print("-" * 30)

    print(f"The best equation found is: {best_equation['equation']}")
    #print(f"Its SymPy format is: {model.sympy()}")
    #print(f"Its score is: {best_equation['score']:.4f}")
    #print("-" * 30)

    # 5. Get the predicted values using the best equation
    # The .predict() method automatically uses the best equation found during the fit.
    y_predicted = model.predict(X)
    # scatterplot of predicted vs actual
    plt.figure()
    plt.scatter(Y_perset[setname], y_predicted, color='blue', alpha=0.5
               , label="Predicted by formula")
    plt.scatter(Y_perset[setname], X_perset[setname][testlabel], 
                color='green', alpha=0.5, label="Actual "+testlabel)
    plt.xlabel("Actual "+labelname)
    plt.ylabel("Predicted "+labelname)
    plt.title("Set: "+setname+" Feature: All")
    plt.plot([min(Y_perset[setname]), max(Y_perset[setname])], [min(Y_perset[setname]), max(Y_perset[setname])], color='red')
    plt.savefig("scatter_predicted_vs_actual_"+setname+".png")
    plt.close()
    mse = mean_squared_error(Y_perset[setname], y_predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(Y_perset[setname], y_predicted)
    mape = mean_absolute_percentage_error(Y_perset[setname], y_predicted)
    metrics[setname]['EQ rmse'] = rmse
    metrics[setname]['EQ r2'] = r2
    metrics[setname]['EQ mape'] = mape

    y_predicted = X_perset[setname][testlabel]
    mse = mean_squared_error(Y_perset[setname], y_predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(Y_perset[setname], y_predicted)
    mape = mean_absolute_percentage_error(Y_perset[setname], y_predicted)
    metrics[setname]['FSPE rmse'] = rmse
    metrics[setname]['FSPE r2'] = r2
    metrics[setname]['FSPE mape'] = mape    


In [ ]:
# histogram of metrics 
#print("Metrics summary: ")
#for setname in supersetnames:
#    print("Set: ", setname)
#    for m in metrics[setname]:
#        print("  ", m, ": ", metrics[setname][m])   
# plot histogram of metrics
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
labels = list(supersetnames)
x = np.arange(len(labels))  # the label locations
width = 0.2  # the width of the bars
fig, ax = plt.subplots(figsize=(10, 6))
#rects1 = ax.bar(x - width, [metrics[setname]['EQ rmse'] for setname in supersetnames], width, label='EQ RMSE')
#rects2 = ax.bar(x, [metrics[setname]['FSPE rmse'] for setname in supersetnames], width, label='FSPE RMSE')
#rects3 = ax.bar(x + width, [metrics[setname]['EQ r2'] for setname in supersetnames], width, label='EQ R2')
#rects4 = ax.bar(x + 2*width, [metrics[setname]['FSPE r2'] for setname in supersetnames], width, label='FSPE R2')
rects5 = ax.bar(x + 3*width, [metrics[setname]['EQ mape'] for setname in supersetnames], width, label='EQ MAPE')
rects6 = ax.bar(x + 4*width, [metrics[setname]['FSPE mape'] for setname in supersetnames], width, label='FSPE MAPE')                
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Metrics by set and method')
ax.set_xticks(x + width / 2)
ax.set_xticklabels(labels)
ax.legend() 
fig.tight_layout()
plt.xticks(rotation=45)  
plt.show()


fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x - width, [metrics[setname]['EQ rmse'] for setname in supersetnames], width, label='EQ RMSE')
rects2 = ax.bar(x, [metrics[setname]['FSPE rmse'] for setname in supersetnames], width, label='FSPE RMSE')
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Metrics by set and method')
ax.set_xticks(x + width / 2)
ax.set_xticklabels(labels)
# rotate xlabel of 45 degree
plt.xticks(rotation=45)  
ax.legend() 
fig.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(10, 6))
rects3 = ax.bar(x + width, [metrics[setname]['EQ r2'] for setname in supersetnames], width, label='EQ R2')
rects4 = ax.bar(x + 2*width, [metrics[setname]['FSPE r2'] for setname in supersetnames], width, label='FSPE R2')
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Metrics by set and method')
ax.set_xticks(x + width / 2)
ax.set_xticklabels(labels)
ax.legend() 
fig.tight_layout()
plt.xticks(rotation=45)  
plt.show()